## Agent Tool Calling

### Initializing LLM

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv ()


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano", api_key=os.getenv ("OPENAI_API_KEY"))

In [ ]:
response = llm.invoke("What is Agentic AI")
response

### Building a simple Graph

In [ ]:
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_core.messages import AnyMessage
from langchain_core.messages import HumanMessage, SystemMessage


sys_prompt = """You are an expert assistante for using VSCode.
You are to give the user guidance on using VScode.
you are to responde to message on VScode only.
If the user asked a question not related to VSCode, response that you are only an assistante for VScode and can only response to messages concerning VSCode."""

sys_msg = SystemMessage (content=sys_prompt)
def llm_call (state:MessagesState) -> MessagesState:
    msg = state['messages']
    response = llm.invoke ([sys_msg] + msg)
    return {'messages': response}

builder = StateGraph (MessagesState)
builder.add_node ('LLM Call', llm_call)

builder.add_edge (START, 'LLM Call')
builder.add_edge ('LLM Call', END)

graph = builder.compile ()

from IPython.display import display, Image

display (Image (graph.get_graph().draw_mermaid_png()))

### Building Tools for an Agent

In [ ]:
from langchain_core.tools import tool

@tool
def multiply (a: int, b: int) -> int:
    """
    Multiply a and b

    Args:
        a: first int
        b: second int
    """
    return a*b

@tool
def add (a: int, b: int) -> int:
    """
    Adds a and b

    Args:
        a: first int
        b: second int
    """
    return a+b


tools = [multiply, add]
llm_with_tool = llm.bind_tools (tools)

### Defining Assistant

In [ ]:
sys_prompt = """You are an expert assistant for performing arithmetic operations.
You have tools for addition, subtraction, multiplication, and division.
You are to respond only to arithmetic-related questions.
If the user asks anything unrelated to arithmetic, reply that you can only assist with arithmetic operations."""


sys_msg = SystemMessage(content= sys_prompt)

def assistant (state: MessagesState) -> MessagesState:
    response = llm_with_tool.invoke ([sys_msg] + state ['messages'])
    return {'messages': response}


from langgraph.prebuilt import tools_condition, ToolNode

builder = StateGraph (MessagesState)
builder.add_node ('tools', ToolNode(tools))
builder.add_node ('assistant', assistant)

builder.add_edge (START, 'assistant')
builder.add_conditional_edges ('assistant', tools_condition)
builder.add_edge ('tools', END)

graph = builder.compile ()

#print (graph.get_graph().draw_mermaid())
display (Image (graph.get_graph().draw_mermaid_png()))

In [ ]:
query = "What can you do for me"
response = graph.invoke (input={'messages': [HumanMessage(content=query)]})

for m in response['messages']:
    m.pretty_print ()

### Making it smart

In [ ]:
builder = StateGraph (MessagesState)
builder.add_node ('tools', ToolNode(tools))
builder.add_node ('assistant', assistant)

builder.add_edge (START, 'assistant')
builder.add_conditional_edges ('assistant', tools_condition)
builder.add_edge ('tools', 'assistant')

graph = builder.compile ()

display (Image( (graph.get_graph().draw_mermaid_png())))

### Web search

pip install tavily

In [ ]:
from tavily import TavilyClient

TavilySearch = TavilyClient (api_key=os.getenv ("TAVILY_API_KEY"))

In [ ]:
searchResult = TavilySearch.search (query="latest Anambra state governorship election results",
                                    max_results=3,)

In [ ]:
searchResult['results']

In [ ]:
searchContent = ""

for result in searchResult['results']:
    content = f"""Source: {result['url']}
Content: {result['content']}\n\n"""
    searchContent += content

print (searchContent)

### ChatBot with realtime websearch

In [ ]:
@tool
def searchTool (query:str) -> str:
    """
    Searches the web in real-time and returns the response.

    Args:
        query: String--A query to search in the web
    """
    searchResult = TavilySearch.search (query=query,
                                    max_results=5,)
    searchContent = ""

    for result in searchResult['results']:
        content = f"""Source: {result['url']}
    Content: {result['content']}\n\n"""
        searchContent += content

    return searchContent

tools = [searchTool]
llm_search = llm.bind_tools (tools=tools)

sys_prompt = """You are an intelligent assistant that can answer any question.
If you know the answer, respond directly.
If you do not know the answer, use your web search tool to find the information in real-time and respond accurately.
Always provide clear and helpful answers based on the information available."""

sys_msg = SystemMessage (content=sys_prompt)

def assistant (state:MessagesState) -> MessagesState:
    msg = state['messages']
    response = llm_search.invoke ([sys_msg] + msg)
    return {'messages': response}


builder = StateGraph (MessagesState)
builder.add_node ('tools', ToolNode(tools))
builder.add_node ('assistant', assistant)

builder.add_edge (START, 'assistant')
builder.add_conditional_edges ('assistant', tools_condition)
builder.add_edge ('tools', 'assistant')

graph = builder.compile ()

display (Image( (graph.get_graph().draw_mermaid_png())))

In [ ]:
query = "Who are youS"
msg = HumanMessage (content= query)

response = graph.invoke (input={'messages': [msg]})

for m in response['messages']:
    m.pretty_print()